In [ ]:
# 找出在cake人力銀行中，搜尋「資料工程師」的職缺資料

import urllib.request as req
import bs4 as bs

import pandas as pd

import time

# 嘗試處理，抓到後面沒資料的話，出現http 404，設定另一組try...except
import requests
from urllib.error import HTTPError

# 準備一個空白的list
table = [] # Clear the table list before scraping

for i in range(100):
    page = i + 1
    # Corrected URL to include page number，需要原本的網址後面加上?page=
    url = "https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=" + str(page)
    print("頁數:", url)
    # 添加了對request清單長度的檢查，以防止任何職位清單未包含所有預期元素時出現潛在的 IndexError 錯誤。
    # 添加了 try...except 程式碼區塊，用於捕獲抓取過程中的任何其他潛在錯誤，並允許程式碼繼續處理後續頁面。

    time.sleep(3)
    try:
        resp = req.urlopen(url)
        content = resp.read()
        html = bs.BeautifulSoup(content)

        # find和find_all是區塊的專屬功能，抓取一個div（區塊）有哪些符合搜尋條件的，而其產出的狀態為 list。
        # js為jobs（多個），j為job（單個）
        js = html.find_all("div", {"class":"JobSearchItem_container__oKoBL"})
        # 以上find_all萃取出 所有職缺區塊
        for j in js:
            # 欲抓取的資料
            t = j.find("a", {"class":"JobSearchItem_jobTitle__bu6yO"})# t為職缺名稱
            c = j.find("a", {"class":"JobSearchItem_companyName__bY7JI"})# c為公司名稱
            request = j.find_all("div", {"class":"InlineMessage_label__LJGjW"})# 表需要的員工條件，有多個資料共用同一個區塊

            # 在訪問所需元素之前，檢查它們是否存在
            if len(request) > 4:
                jo = request[0]# jo為需求人數，job opening
                l = request[1]# l為工作地點
                s_text = request[2].get_text().strip()# s為薪資範圍
                ex_text = request[3].get_text().strip()# ex為工作年資
                m = request[4]# m為管理責任
            else:
                # 處理某些元素缺失的情況
                jo = None
                l = None
                s_text = None
                ex_text = None
                m = None

            # m = resquest[4]# m為管理責任，此階段出現「IndexError: list index out of range」
            jd = j.find("div",{"class":"JobSearchItem_description__si5zg"})# Jd為工作說明



            # 欲產生的資料
            t_text = t.get_text().strip() if t else None
            c_text = c.get_text().strip() if c else None
            jo_text = jo.get_text().strip() if jo else None
            l_text = l.get_text().strip() if l else None
            # s = s.get_text().strip() 原本設定要做，但系統直接將text設定在上面，可忽略
            # ex = ex.get_text().strip() 原本設定要做，但系統直接將text設定在上面，可忽略
            m = m.get_text().strip() if m else None
            jd_text = jd.get_text().strip() if jd else None
            t_href = "www.cake.me/" + t["href"] if t and "href" in t.attrs else None

            # 一筆資料就是一個字典
            row = {
                "職務名稱:":t_text,
                "公司名稱:":c_text,
                "需求人數:":jo_text,
                "工作地點:":l_text,
                "薪資範圍:":s_text,
                "工作年資:":ex_text,
                "管理責任:":m,
                "工作說明:":jd_text,
                "網站連結:":t_href,
            }
            table.append(row)

            # 呈現的資料（前測檢視用）
            # print("職務名稱:",t_text)
            # print("公司名稱:",c_text)
            # print("需求人數:",jo_text)
            # print("工作地點:",l_text)
            # print("薪資範圍:",s)
            # print("工作年資:",ex)
            # print("管理責任:",m)
            # print("工作說明:",jd_text)
            # print("網站連結:",t_href)
            # 分隔線
            # print("-" * 30)
    except Exception as e:
        print(f"Error processing page {page}: {e}")
        # 如果出現錯誤，請繼續下一頁


    # 嘗試處理，抓到後面沒資料的話，出現http 404
    try:
        response = requests.get(url)
        response.raise_for_status()  # 若不是200，會觸發HTTPError
        data = response.text

        # 若頁面有明顯「無資料」訊息，也可以加條件判斷
        if "No jobs found" in data or len(data) < 500:
            print(f"第 {page} 頁沒有資料，停止爬取。")
            break

        print(f"成功抓取第 {page} 頁")
        # 這裡放解析與儲存資料的程式碼
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"第 {page} 頁不存在（404），停止爬取。")
            break
        else:
            print(f"第 {page} 頁發生錯誤：{e}")
            continue  # 其他錯誤則跳過這頁

# 轉換存檔
df = pd.DataFrame(table)
df.to_csv("cakejob.csv", encoding="utf-8")
df

頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=1
成功抓取第 1 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=2
成功抓取第 2 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=3
成功抓取第 3 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=4
成功抓取第 4 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=5
成功抓取第 5 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=6
成功抓取第 6 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=7
成功抓取第 7 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=8
成功抓取第 8 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=9
成功抓取第 9 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?page=10
成功抓取第 10 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?pag

職務名稱:  \
0                                                資料工程師   
1                                Data Engineer / 資料工程師   
2               【風控資料工程師】Data Engineer(Risk Analytics)   
3                        數據工程師實習生 Data Engineer Intern   
4                                        Data Engineer   
..                                                 ...   
568                   Saker Squad | 軟體 × Embedded 雙棲開發   
569                         Junior .NET Core Developer   
570      🍰【女生聲優實習生 Voice Actor Intern】— 加入我們，讓聲音為品牌說話！   
571                                Security Researcher   
572  【毛小愛】保姆夥伴經理 Sitter Success Manager  #寵物友善 #一日遠...   

                               公司名稱:                         需求人數:  \
0    群健科技股份有限公司Cofit Healthcare Inc.  2・Full-time・Mid-Senior level   
1                               SWAG  1・Full-time・Mid-Senior level   
2                        智鈦星平台服務有限公司       1・Full-time・Entry level   
3               COMMEET＿擁樂數據服務股份有限公司      1・Internship・Entry level   
4                             Pinkoi                          None   
..                               ...                           ...   
568                       神準科技股份有限公司       5・Full-time・Entry level   
569                         創科數碼有限公司       1・Full-time・Entry level   
570                       深智科技股份有限公司       1・Internship・Internship   
571                          MaiCoin  1・Full-time・Mid-Senior level   
572                     可艾創意科技股份有限公司  1・Full-time・Mid-Senior level   

                                                 工作地點:  \
0                                  Taipei City, Taiwan   
1                                  Taipei City, Taiwan   
2                                  Taipei City, Taiwan   
3    Songshan District, Taipei City, Taiwan 105・Tai...   
4                                                 None   
..                                                 ...   
568  Nangang District, Taipei City, Taiwan・Guishan ...   
569                                             Taiwan   
570                                 New Taipei, Taiwan   
571                Xinyi District, Taipei City, Taiwan   
572                                Taipei City, Taiwan   

                      薪資範圍:                                           工作年資:  \
0    70K ~ 100K TWD / month                  4 years of experience required   
1                Negotiable                  3 years of experience required   
2    60K ~ 120K TWD / month                  2 years of experience required   
3      200 ~ 250 TWD / hour  No requirement for relevant working experience   
4                      None                                            None   
..                      ...                                             ...   
568   80K ~ 90K TWD / month  No requirement for relevant working experience   
569   55K ~ 70K TWD / month                  2 years of experience required   
570    250 ~ 500 TWD / hour  No requirement for relevant working experience   
571      1M ~ 2M TWD / year                  3 years of experience required   
572   40K ~ 70K TWD / month                  3 years of experience required   

                                     管理責任:  \
0             No management responsibility   
1             No management responsibility   
2             No management responsibility   
3             No management responsibility   
4                                     None   
..                                     ...   
568           No management responsibility   
569           No management responsibility   
570           No management responsibility   
571           No management responsibility   
572  Managing staff numbers: not specified   

                                                 工作說明:  \
0    ★ 關於 Cofit\nHi，我們是 Cofit，正在打造一個橫跨台灣、美加、新加坡、香港、...   
1    DescriptionAt SWAG Live, data is the backbone ...   
2    身為風控數據工程師，你將負責設計、建置與維運公司的數據基礎設施（Data Platform）...   
3    COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...   
4    【關於 Pinkoi Data S

In [ ]:
job_title_distribution = df['職務名稱:'].value_counts()
display(job_title_distribution)

職務名稱:
Data Engineer                                    4
後端工程師                                            3
前端工程師                                            2
Senior - Software Engineer                       2
Software Engineering Manager II, Google Cloud    2
                                                ..
Engineering Manager, Core Data Processing        1
Senior Engineering Manager, Data Discovery       1
Firmware Engineer, Pixel Power                   1
Senior Research Engineer, AI/ML                  1
【LiMI國際智能製造人才計畫】TE產品測試工程師_CIPS（社會新鮮人、應屆畢業生專區）    1
Name: count, Length: 562, dtype: int64

In [ ]:
df = pd.DataFrame(table)
display(df.head())

Empty DataFrame
Columns: []
Index: []

In [ ]:
pip install selenium beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 32.5 MB/s eta 0:00:00
